In [2]:
import os
import sys
sys.path.append(os.path.join(os.getcwd().replace("notebooks/large_models", "")))

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from config import Config
from data.dataset import NetworkDataset, load_datasets
from model.mlp import MLP
from model_config import MLP_Models, LSTM_Models, GRU_Models, CNN_models
from utils.benchmark import Benchmark

In [3]:
conf = Config()
load_model = CNN_models()
model_conf = load_model.cnn_4
model = load_model.get_model(model_conf)
model.load()
print(conf.device)

Checkpoint loaded from /home/ubuntu/Network-Packet-ML-Model/checkpoint/large_model/cnn_4.pth!
cpu


In [4]:
# data loader
X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, model_type=load_model.type)

# create train, val and test dataloaders
train_dataset = NetworkDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, conf.batch_size, shuffle=True)

val_dataset = NetworkDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, conf.batch_size, shuffle=True)

dataset = NetworkDataset(X_test, y_test)
loader = DataLoader(dataset, conf.batch_size, shuffle=True)

In [5]:
name = model_conf["name"]
result_path = os.path.join(conf.benchmark_dpu, "large_model", name + ".txt")

# run benchmark
benchmark = Benchmark(model, loader, conf.batch_size, name, result_path)
benchmark()

# print and save result
benchmark.print_result()
benchmark.save()

[W115 23:07:19.915345226 CPUAllocator.cpp:245] Memory block of unknown size was allocated before the profiling started, profiler results will not include the deallocation event


Benchmark - cnn_4 model:

Memory usage (MB):
Avg memory usage: 0.946MB
Peak memory usage: 21.214MB

Model inference latency on one batch (batch size = 1):
Avg latency: 11.298ms
Min latency: 10.511ms
Max latency: 23.446ms

Model inference throughput (batch size = 1):
Throughput: 39.98 samples/sec

Model inference CPU usage (number of logical cores) during runtime:
CPU runtime: 1.19 seconds
Average CPU usage: 3.52/8 cores

Model (cnn_4) Macro-F1, Micro-F1 and Macro ROC AUC scores:
Macro-F1 score: 0.51
Micro-F1 score: 0.89
Macro ROC AUC score: nan





/home/ubuntu/Network-Packet-ML-Model/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1201: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/ubuntu/Network-Packet-ML-Model/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1201: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/ubuntu/Network-Packet-ML-Model/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1201: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/ubuntu/Network-Packet-ML-Model/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1201: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/ubuntu/Network-Packet-ML-Model/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1201: UndefinedMetri